In [55]:
import pandas as pd
import numpy as np
from datetime import datetime

src_train = pd.read_csv('training_set.csv', delimiter=';')
# Save predictions
src_score_org = pd.read_csv('test_set.csv', delimiter=';')
src_score = pd.read_csv('test_set.csv', delimiter=';')

def runDataManipulation(df):
    # Data manipulation
    for col in ['fecha_reserva', 'check-in', 'check-out']:
      df[col] = df[col].apply(lambda d: datetime.strptime(d, '%Y-%m-%d'))  

def computeColumns(df):
# Computed columns
    df['release'] = df.apply(lambda r: (r['check-in'] - r['fecha_reserva']).days, axis=1)
    #df['bookmonth'] = df['fecha_reserva'].apply(lambda d: d.month)
    #df['checkinmonth'] = df['check-in'].apply(lambda d: d.month)

for df in [src_train, src_score]:  
    runDataManipulation(df)
    computeColumns(df)

msk = np.random.rand(len(src_train)) < 0.8
df_train = src_train[msk]
df_test = src_train[~msk]
print("Train %d test %d" % (len(df_train), len(df_test)))
df_train.head()





Train 21836 test 5532


,index,fecha_reserva,pais_cliente,importe_reserva,roomnigths,check-in,check-out,is_reserva_cancelada,total_habitaciones,hotel,localizador,nombre_cliente,telefono_cliente,email_cliente,release
0,0,2017-06-01,Costa Rica,0.003693,3,2017-07-21,2017-07-24,1,1,02bdf29aa2f019efff360097aea1c69e48904c9f,445ddc89c2481d5ba706290f56d684474f2ea348,353e746e9761df792d016fa60511ea5af2c5fe14,ccf60c41f8826c6f66a2d99656c80bfeab1520e1,c0d965fb91f6211a648ceeaf5e2d4af662afbc32,50
1,1,2017-06-01,España,0.008979,7,2017-06-11,2017-06-18,1,1,d46b813cd813e2318a35509103b1f081db5f77a3,a5af59e5353d2aa9a1ad1af0efc2c705f51081af,c2fc06233869c2c9485b3f8705483308c6fb34fa,aa0b0c3c980ffd82b0065d75e94ec2c8393ef1d4,a0ce353f75bd324baae267a698304c75df508557,10
2,2,2017-06-01,Reino Unido,0.004807,2,2017-06-16,2017-06-18,0,1,5e15fec283ff9ceb8449e558d8c5c81ac9fd2cfc,810f16cec64559f3bd15cba281e750d0561324bd,314f1860d9eac211285d704c56d053ed60726dc2,b7b46949047ab5bc72b6f653bcbb4cbe001e940f,4d2eaf0b2bdee0e348b9a735d6bf8cf27416b0ff,15
5,5,2017-06-01,República Dominicana,0.012608,6,2017-06-29,2017-07-02,0,2,290d5f265e3662e4df4e4e4a90f1612cc68404f8,0d39566a4b888d10bb722891bbf3c6bd289066c4,39a6d2497a61d62fb0d3bfca033c22cf69b40b88,f9ee45fc184944a910332c5177657418a75a6eae,657c22b1e3b43728df4f4a8e670e3971fad2105d,28
6,6,2017-06-01,España,0.004959,4,2017-09-10,2017-09-14,1,1,28894d8f7f388a274b0bde507e15bb3d4cbbbb3f,d0cb6623a44f433148e517f84fb03d010d650845,8d324b5a1b93bf146f15f617160e1bd0ebf35cd5,f8f1eefccc849c8fae2865ce599aeec89886ab60,e6a93e51ed788ffc827e033c4373dca11aaa1cf8,101


In [56]:
from auto_ml import Predictor

# Tell auto_ml which column is 'output'
# Also note columns that aren't purely numerical
# Examples include ['nlp', 'date', 'categorical', 'ignore']
column_descriptions = {
    'index':'ignore',
    'localizador': 'ignore',
    #'nombre_cliente': 'ignore',
    #'telefono_cliente': 'ignore',
    #'email_cliente': 'ignore',
    'is_reserva_cancelada': 'output',
    'pais_cliente': 'categorical',
    'hotel': 'categorical',
    'localizador': 'categorical',
    'nombre_cliente': 'categorical',
    'telefono_cliente': 'categorical',
    'email_cliente': 'categorical',
    'fecha_reserva': 'date',
    'check-in': 'date',
    'check-out': 'date'
}

# Can pass in type_of_estimator='regressor' as well
ml_predictor = Predictor(type_of_estimator='classifier', column_descriptions=column_descriptions)

# Wait for the machine to learn all the complex and beautiful patterns in your data...
ml_predictor.train(df_train, ml_for_analytics=True)

# Score the model on test data
# test_score = ml_predictor.score(df_test, df_test.MEDV)

# .predict and .predict_proba take in either:
# A pandas DataFrame
# A list of dictionaries
# A single dictionary (optimized for speed in production evironments)
predictions = ml_predictor.predict(df_test)



Welcome to auto_ml! We're about to go through and make sense of your data using machine learning, and give you a production-ready pipeline to get predictions with.

If you have any issues, or new feature ideas, let us know at https://github.com/ClimbsRocks/auto_ml
Now using the model training_params that you passed in:
{}
After overwriting our defaults with your values, here are the final params that will be used to initialize the model:
{'presort': False, 'learning_rate': 0.1, 'warm_start': True}
Running basic data cleaning
Performing feature scaling
Fitting DataFrameVectorizer
Now using the model training_params that you passed in:
{}
After overwriting our defaults with your values, here are the final params that will be used to initialize the model:
{'presort': False, 'learning_rate': 0.1, 'warm_start': True}


********************************************************************************************
About to fit the pipeline for the model GradientBoostingClassifier to predict is_

[223] random_holdout_set_from_training_data's score is: -0.188
[226] random_holdout_set_from_training_data's score is: -0.187
[229] random_holdout_set_from_training_data's score is: -0.187
[232] random_holdout_set_from_training_data's score is: -0.187
[235] random_holdout_set_from_training_data's score is: -0.187
[238] random_holdout_set_from_training_data's score is: -0.187
[241] random_holdout_set_from_training_data's score is: -0.187
[244] random_holdout_set_from_training_data's score is: -0.187
[247] random_holdout_set_from_training_data's score is: -0.187
[250] random_holdout_set_from_training_data's score is: -0.187
[255] random_holdout_set_from_training_data's score is: -0.187
[260] random_holdout_set_from_training_data's score is: -0.187
[265] random_holdout_set_from_training_data's score is: -0.187
[270] random_holdout_set_from_training_data's score is: -0.187
[275] random_holdout_set_from_training_data's score is: -0.187
[280] random_holdout_set_from_training_data's score is:

Calculating feature responses, for advanced analytics.


C:\Users\gustavo.brian\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


The printed list will only contain at most the top 100 features.
The full analytics results will be saved to a filed called: auto_ml_analytics_results_is_reserva_cancelada.csv

+---------+-----------------------------------------------------------+--------------+----------+-------------------+-------------------+-----------+-----------+-----------+-----------+
|         | Feature Name                                              |   Importance |    Delta |   FR_Decrementing |   FR_Incrementing |   FRD_abs |   FRI_abs |   FRD_MAP |   FRI_MAP |
|---------+-----------------------------------------------------------+--------------+----------+-------------------+-------------------+-----------+-----------+-----------+-----------|
| 99.0000 | telefono_cliente=6ee4c8bff99a9f3de6d128990a3bcde574fdf387 |       0.0012 | nan      |          nan      |          nan      |  nan      |  nan      |  nan      |  nan      |
| 98.0000 | nombre_cliente=88fb2cbf49eb39c22cc89f76f59970fe6a057095   |       0

In [57]:
# Where new_data is a single dictionary, or a DataFrame
predictions = ml_predictor.predict(df_test)
df_test['prediction'] = predictions 
df_test['match'] = df_test['prediction'] == df_test['is_reserva_cancelada'] 
classes = [[i,k] for i,k in enumerate(df_test['is_reserva_cancelada'])]
validation = list(map(lambda p: [p[1],predictions[p[0]],p[1] == predictions[p[0]]], classes))
print(validation[0:5])
total = len(validation)
ok = len(list(filter(lambda k: k[2], validation)))
perc = 100 / total * ok
print("Validation %d of %d - %f%%" % (ok, total, perc))
df_test.head()

[[0, 0, True], [1, 0, False], [1, 0, False], [1, 1, True], [0, 0, True]]
Validation 3996 of 5532 - 72.234273%


,index,fecha_reserva,pais_cliente,importe_reserva,roomnigths,check-in,check-out,is_reserva_cancelada,total_habitaciones,hotel,localizador,nombre_cliente,telefono_cliente,email_cliente,release,prediction,match
3,3,2017-06-01,España,0.001905,2,2017-06-11,2017-06-13,0,1,3c0e030d4ab643f402af566b86266cbaf6a610ef,63f9e567275d9d767c2772670b49d9a3c44a5be0,a12ee97fd2b7c328c7f32bf6b525d822a04a8738,a0460d93425231616ed74397a2cb16fda09350a4,a0ce353f75bd324baae267a698304c75df508557,10,0,True
4,4,2017-06-01,España,0.004168,4,2017-09-10,2017-09-14,1,1,d46b813cd813e2318a35509103b1f081db5f77a3,363a04550045efd2d4fea70117b80620081c3746,8d324b5a1b93bf146f15f617160e1bd0ebf35cd5,f8f1eefccc849c8fae2865ce599aeec89886ab60,19a229a6aeb9d7e3d50df5428df48321ae5de6f8,101,0,False
14,15,2017-06-01,España,0.009529,6,2017-10-18,2017-10-24,1,1,e7835ec4284c7402b3cb975860af6b8f6d377505,41acec97a479a55724252bee1329ae5ed0fe8854,ecd48dd1f0c2c79f27d2f06489cf3f5f682913f1,4cbc4497f72208d6573396aecbfe6b99abe7264c,a0ce353f75bd324baae267a698304c75df508557,139,0,False
16,17,2017-06-01,Puerto Rico,0.014588,9,2017-10-12,2017-10-15,1,3,db37f5f937e05616741ceaf9af8d48f6810be724,2d6f2d5cb96a3f97db6e55579ff4f6ad715daa58,2952d180bea077ff81a3277e00596006590dcc44,8a80267673bec3257ac74191f2c896bec17ff932,d303750b180e99bfcd427c1843c3a03cf5f99f29,133,1,True
17,19,2017-06-01,España,0.005610,4,2017-06-19,2017-06-23,0,1,d46b813cd813e2318a35509103b1f081db5f77a3,6fce19acca68703304ede2fe540981394b8d6776,6a358bedb7a49aa33ea4d161515154822539564c,f7e14efcf3383844b9edacf3fb88523f88b987ae,955788dfcc890bb336a23bfbd0b2163a9b5f4dcf,18,0,True


In [61]:
# Where new_data is a single dictionary, or a DataFrame
predictions = ml_predictor.predict_proba(src_score)
src_score_org['probabilidad'] = predictions
# print(len(list(filter(lambda x: x != 0, predictions))))
src_score_org.head()
import time
millis = int(round(time.time() * 1000))
fname = 'gustavobrianA%dProba_test_%d.csv'  %(int(perc), millis)
src_score_org.to_csv(fname, columns = ['index','probabilidad'], index=False)

In [59]:
src_eval = pd.read_csv('eval_set.csv', delimiter=';')
runDataManipulation(src_eval)
computeColumns(src_eval)
src_eval.head()

,index,fecha_reserva,pais_cliente,importe_reserva,roomnigths,check-in,check-out,total_habitaciones,hotel,localizador,nombre_cliente,telefono_cliente,email_cliente,release
0,12919,2017-06-12,España,0.001392,1,2017-06-24,2017-06-25,1,231da18e12b7782225051a6400c588fcb8d71c40,94efbe8c1a88253fc89a45d11d7b16281f5d0275,3929c72f7ac535d0f435aa16bc0e16452b44644d,f32fe131936ba489026d0ee9115391c87b25b610,f91eaae6dbfb4865ba1dc3e071cb6b5ffa2fb150,12
1,12920,2017-06-12,España,0.008720,7,2017-08-06,2017-08-13,1,bf060d97eccf35431001cee753487e3c969ff14b,7c970948fbe8999a30da11869597621bef7bd796,7b238f28c2ee5cc602f8b35f659f24d5bd8342d9,9ca050ad0979c583b0fc10b1e3bf8c8c1f15ba8b,ec01a617a9b2765fabe68d6120c9071c4da6e30e,55
2,12921,2017-06-12,España,0.004250,9,2017-07-23,2017-08-01,1,1fd43b576b8dcdb109de2af1c6741febc1319845,55b1de175596f1289b31897779ed0f371188c444,94b1de9e214d789446309c379b6eedcfb8fd5722,546bdead823d06d016a9a5371369db5d886bd953,0ba05960069eb29f66e9a777890535893779aafb,41
3,12922,2017-06-12,España,0.007232,7,2017-09-08,2017-09-15,1,5046d28b88ad50aa40407d6ef7fa30bae007fa15,ccb49769f051c66c951556df7693bb7295b923d8,0fc35465b3fec7863d7b36d3138049d80b99c0a1,0177016fdeee8510b57c33f0cd6797b84a108c2f,dfb73ce70cd7658c03d74bf80647953ed1777eb7,88
4,12923,2017-06-12,España,0.008359,7,2017-09-14,2017-09-21,1,90f4adbe1fb02c94b65af630d56f45f99b5cf7d8,067f83181d21cd55514a7ef2a6a8e8327f8c1943,3b3314c0db3ef57f20839dbe317529625e1a2637,b469a93765fef991b861dbe6cf4dd3d800bbc3ae,5295ddc64e82113497049e9226f87810e5c4ae4b,94


In [62]:
# Where new_data is a single dictionary, or a DataFrame
predictions = ml_predictor.predict_proba(src_eval)
src_eval['probabilidad'] = predictions
fname = 'gustavobrian_eval.csv'
src_eval.to_csv(fname, columns = ['index','probabilidad'], index=False)
src_eval.head()


,index,fecha_reserva,pais_cliente,importe_reserva,roomnigths,check-in,check-out,total_habitaciones,hotel,localizador,nombre_cliente,telefono_cliente,email_cliente,release,probabilidad
0,12919,2017-06-12,España,0.001392,1,2017-06-24,2017-06-25,1,231da18e12b7782225051a6400c588fcb8d71c40,94efbe8c1a88253fc89a45d11d7b16281f5d0275,3929c72f7ac535d0f435aa16bc0e16452b44644d,f32fe131936ba489026d0ee9115391c87b25b610,f91eaae6dbfb4865ba1dc3e071cb6b5ffa2fb150,12,0.709844
1,12920,2017-06-12,España,0.008720,7,2017-08-06,2017-08-13,1,bf060d97eccf35431001cee753487e3c969ff14b,7c970948fbe8999a30da11869597621bef7bd796,7b238f28c2ee5cc602f8b35f659f24d5bd8342d9,9ca050ad0979c583b0fc10b1e3bf8c8c1f15ba8b,ec01a617a9b2765fabe68d6120c9071c4da6e30e,55,0.561374
2,12921,2017-06-12,España,0.004250,9,2017-07-23,2017-08-01,1,1fd43b576b8dcdb109de2af1c6741febc1319845,55b1de175596f1289b31897779ed0f371188c444,94b1de9e214d789446309c379b6eedcfb8fd5722,546bdead823d06d016a9a5371369db5d886bd953,0ba05960069eb29f66e9a777890535893779aafb,41,0.551696
3,12922,2017-06-12,España,0.007232,7,2017-09-08,2017-09-15,1,5046d28b88ad50aa40407d6ef7fa30bae007fa15,ccb49769f051c66c951556df7693bb7295b923d8,0fc35465b3fec7863d7b36d3138049d80b99c0a1,0177016fdeee8510b57c33f0cd6797b84a108c2f,dfb73ce70cd7658c03d74bf80647953ed1777eb7,88,0.468262
4,12923,2017-06-12,España,0.008359,7,2017-09-14,2017-09-21,1,90f4adbe1fb02c94b65af630d56f45f99b5cf7d8,067f83181d21cd55514a7ef2a6a8e8327f8c1943,3b3314c0db3ef57f20839dbe317529625e1a2637,b469a93765fef991b861dbe6cf4dd3d800bbc3ae,5295ddc64e82113497049e9226f87810e5c4ae4b,94,0.459506
